In [30]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
import catboost as cb

In [31]:
train = pd.read_csv("Dataset/baseline_data_imputed.csv").drop(["id_jalan", "id_titik_mulai", "id_titik_akhir", "hour_group"], axis=1)

y = train["rerata_kecepatan"]
train.drop("rerata_kecepatan",axis=1, inplace=True)
train["rerata_kecepatan"] = y


test = pd.read_csv("Dataset/test_with_ext_no_ohe.csv")
ids = pd.read_csv("Dataset/test.csv")["id"]

In [32]:
mapping = {
    1: "Saturday",
    2: "Sunday",
    3: "Monday",
    4: "Tuesday",
    5: "Wednesday",
    6: "Thursday",
    7: "Friday",
    8: "Saturday",
    9: "Sunday",
    10: "Monday",
    11: "Tuesday",
    12: "Wednesday",
    13: "Thursday",
    14: "Friday",
    15: "Saturday",
    16: "Sunday",
    17: "Monday",
    18: "Tuesday",
    19: "Wednesday",
    20: "Thursday",
    21: "Friday",
    22: "Saturday",
    23: "Sunday",
    24: "Monday",
    25: "Tuesday",
    26: "Wednesday",
    27: "Thursday",
    28: "Friday",
    29: "Saturday"
}



test["Day"] = test["Day"].map(mapping)

def is_weekend(day):
    return 1 if day in ['Saturday', 'Sunday'] else 0

test['is_weekend'] = test['Day'].apply(is_weekend)

In [33]:
# y = train["rerata_kecepatan"]
# train, test = train.align(test, join='inner', axis=1)
# train["rerata_kecepatan"] = y

In [34]:
train.head()

,Date,Hour,road_type,node_awal_ways,node_akhir_ways,road_relation,road_node,max_speed,type_highway,id_jalan_id_titik_mulai_id_titik_akhir,is_weekend,hari,rerata_kecepatan
0,1,0,Road,3.0,3.0,13.0,5.0,30.0,trunk,691007296_21390008_1425033102,True,Sabtu,29.126
1,1,1,Road,3.0,3.0,13.0,5.0,30.0,trunk,691007296_21390008_1425033102,True,Sabtu,29.126
2,1,2,Road,3.0,3.0,13.0,5.0,30.0,trunk,691007296_21390008_1425033102,True,Sabtu,34.772
3,1,3,Road,3.0,3.0,13.0,5.0,30.0,trunk,691007296_21390008_1425033102,True,Sabtu,34.161
4,1,4,Road,3.0,3.0,13.0,5.0,30.0,trunk,691007296_21390008_1425033102,True,Sabtu,39.383


In [35]:
test.head()

,Date,Hour,road_type,node_awal_ways,node_akhir_ways,road_relation,road_node,max_speed,type_highway,Day,id_jalan_id_titik_mulai_id_titik_akhir,node_awal_ways_minus_node_akhir_ways,is_weekend
0,23,0,West,3.0,3.0,9,7,30,trunk,Friday,4004732_32046542_6454026544,0.0,0
1,23,0,Side,2.0,2.0,13,11,30,trunk,Friday,182210371_1314925464_1314925496,0.0,0
2,23,0,Lane,2.0,2.0,4,62,20,secondary,Friday,22932408_1482086782_26481020,0.0,0
3,23,0,Side,2.0,2.0,13,11,30,trunk,Friday,182210371_3892883_267337489,0.0,0
4,23,0,Road,2.0,3.0,25,34,30,trunk,Friday,66924592_266041030_2592978110,-1.0,0


In [5]:
ids[:5]

0    0
1    1
2    2
3    3
4    4
Name: id, dtype: int64

In [6]:
cat_columns = ["road_type", "type_highway", "id_jalan_id_titik_mulai_id_titik_akhir"]
cat_columns

['road_type', 'type_highway', 'id_jalan_id_titik_mulai_id_titik_akhir']

In [7]:
X = train.drop("rerata_kecepatan", axis=1)
y = train["rerata_kecepatan"]

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

In [9]:
# for reproducibility
seed_value = 42
np.random.seed(seed_value)

clf = cb.CatBoostRegressor(
    loss_function='MAPE',
    task_type="GPU",
    random_seed=42
)

# Define the hyperparameters grid for tuning
param_grid = {
    'iterations': [1500, 2000, 2500],  # Number of boosting iterations
    'learning_rate': [0.5, 0.7, 0.9],  # Learning rate
    'depth': [6, 8, 10],  # Tree depth
}

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=3, scoring='neg_mean_absolute_percentage_error', verbose=100)

# Perform hyperparameter tuning with cross-validation
grid_search.fit(X_train, y_train, cat_features=cat_columns, eval_set=(X_val, y_val), verbose=100, early_stopping_rounds=50)

# Get the best model and its parameters
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

# Print the results
print('Best CatBoost model parameters:')
print(best_params)
print('CatBoost model is fitted: ' + str(best_model.is_fitted()))
print('CatBoost model parameters:')
print(best_model.get_params())

Fitting 3 folds for each of 27 candidates, totalling 81 fits
[CV 1/3; 1/27] START depth=6, iterations=1500, learning_rate=0.5................
0:	learn: 0.2346856	test: 0.2381213	best: 0.2381213 (0)	total: 20ms	remaining: 30s
100:	learn: 0.2125477	test: 0.2154129	best: 0.2154129 (100)	total: 1.76s	remaining: 24.5s
200:	learn: 0.1944426	test: 0.1967925	best: 0.1967925 (200)	total: 3.43s	remaining: 22.2s
300:	learn: 0.1802273	test: 0.1820776	best: 0.1820776 (300)	total: 5.33s	remaining: 21.3s
400:	learn: 0.1686961	test: 0.1699540	best: 0.1699540 (400)	total: 7.28s	remaining: 20s
500:	learn: 0.1596956	test: 0.1605156	best: 0.1605156 (500)	total: 9.13s	remaining: 18.2s
600:	learn: 0.1526306	test: 0.1530938	best: 0.1530938 (600)	total: 11s	remaining: 16.4s
700:	learn: 0.1470151	test: 0.1472216	best: 0.1472216 (700)	total: 12.9s	remaining: 14.7s
800:	learn: 0.1423558	test: 0.1423077	best: 0.1423077 (800)	total: 15s	remaining: 13.1s
900:	learn: 0.1385105	test: 0.1382390	best: 0.1382390 (900)	t

In [11]:
"""
Best CatBoost model parameters:
{'depth': 8, 'iterations': 1500, 'learning_rate': 0.5}
CatBoost model is fitted: True
CatBoost model parameters:
{'iterations': 1500, 'learning_rate': 0.5, 'depth': 8, 'loss_function': 'MAPE', 'random_seed': 42, 'task_type': 'GPU'}

Best CatBoost model parameters:
{'depth': 10, 'iterations': 2500, 'learning_rate': 0.9}
CatBoost model is fitted: True
CatBoost model parameters:
{'iterations': 2500, 'learning_rate': 0.9, 'depth': 10, 'loss_function': 'MAPE', 'random_seed': 42, 'task_type': 'GPU'}

"""

"\nBest CatBoost model parameters:\n{'depth': 8, 'iterations': 1500, 'learning_rate': 0.5}\nCatBoost model is fitted: True\nCatBoost model parameters:\n{'iterations': 1500, 'learning_rate': 0.5, 'depth': 8, 'loss_function': 'MAPE', 'random_seed': 42, 'task_type': 'GPU'}\n\nBest CatBoost model parameters:\n{'depth': 10, 'iterations': 2500, 'learning_rate': 0.9}\nCatBoost model is fitted: True\nCatBoost model parameters:\n{'iterations': 2500, 'learning_rate': 0.9, 'depth': 10, 'loss_function': 'MAPE', 'random_seed': 42, 'task_type': 'GPU'}\n\n"

In [19]:
clf = cb.CatBoostRegressor(
    iterations=1500,
    verbose=100,
    depth=8, 
    learning_rate=0.5,
    loss_function="MAPE",
    task_type="GPU"
)

clf.fit(
    X_train, y_train,
    cat_features=cat_columns,
    eval_set=(X_val, y_val),
)

0:	learn: 0.2345077	test: 0.2380992	best: 0.2380992 (0)	total: 34.4ms	remaining: 51.5s
100:	learn: 0.2107068	test: 0.2139521	best: 0.2139521 (100)	total: 3.05s	remaining: 42.3s
200:	learn: 0.1908393	test: 0.1936467	best: 0.1936467 (200)	total: 6.26s	remaining: 40.5s
300:	learn: 0.1748961	test: 0.1772095	best: 0.1772095 (300)	total: 9.62s	remaining: 38.3s
400:	learn: 0.1624436	test: 0.1642943	best: 0.1642943 (400)	total: 12.9s	remaining: 35.3s
500:	learn: 0.1524978	test: 0.1538403	best: 0.1538403 (500)	total: 16.3s	remaining: 32.5s
600:	learn: 0.1447441	test: 0.1456655	best: 0.1456655 (600)	total: 19.5s	remaining: 29.2s
700:	learn: 0.1383615	test: 0.1389776	best: 0.1389776 (700)	total: 22.8s	remaining: 25.9s
800:	learn: 0.1330548	test: 0.1333988	best: 0.1333988 (800)	total: 26.3s	remaining: 22.9s
900:	learn: 0.1288393	test: 0.1289905	best: 0.1289905 (900)	total: 29.7s	remaining: 19.8s
1000:	learn: 0.1254740	test: 0.1254865	best: 0.1254865 (1000)	total: 33s	remaining: 16.5s
1100:	learn: 

In [54]:
X_train = train.drop("rerata_kecepatan", axis=1)
y_train = train["rerata_kecepatan"]
X_test = test

In [55]:
X_train.head()

,Date,Hour,road_type,node_awal_ways,node_akhir_ways,road_relation,road_node,max_speed,type_highway,Day,id_jalan_id_titik_mulai_id_titik_akhir,node_awal_ways_minus_node_akhir_ways
0,1,1,Road,3.0,3.0,13,5,30,trunk,6,691007296_21390008_1425033102,0.0
1,1,1,Road,3.0,2.0,10,53,30,primary,6,47010584_1677092762_579493410,1.0
2,1,1,Lane,3.0,2.0,4,62,20,secondary,6,22932408_26486694_1930267566,1.0
3,1,1,Hill,2.0,2.0,11,12,20,primary,6,142479648_1111592522_3775231113,0.0
4,1,1,Road,2.0,2.0,4,3,30,primary,6,8504977_5940503398_5940503394,0.0


In [56]:
X_test.head()

,Date,Hour,road_type,node_awal_ways,node_akhir_ways,road_relation,road_node,max_speed,type_highway,Day,id_jalan_id_titik_mulai_id_titik_akhir,node_awal_ways_minus_node_akhir_ways
0,23,0,West,3.0,3.0,9,7,30,trunk,7,4004732_32046542_6454026544,0.0
1,23,0,Side,2.0,2.0,13,11,30,trunk,7,182210371_1314925464_1314925496,0.0
2,23,0,Lane,2.0,2.0,4,62,20,secondary,7,22932408_1482086782_26481020,0.0
3,23,0,Side,2.0,2.0,13,11,30,trunk,7,182210371_3892883_267337489,0.0
4,23,0,Road,2.0,3.0,25,34,30,trunk,7,66924592_266041030_2592978110,-1.0


In [57]:
y_train[:5]

0    29.126
1    46.576
2    36.587
3    34.063
4    38.336
Name: rerata_kecepatan, dtype: float64

In [58]:
clf = cb.CatBoostRegressor(
    iterations=2000,
    verbose=100,
    depth=8, 
    learning_rate=0.5,
    loss_function="MAPE",
    task_type="GPU"
)

clf.fit(
    X_train, y_train,
    cat_features=cat_columns
)

0:	learn: 0.2352260	total: 43.2ms	remaining: 1m 26s
100:	learn: 0.2114283	total: 3.65s	remaining: 1m 8s
200:	learn: 0.1911840	total: 7.18s	remaining: 1m 4s
300:	learn: 0.1751672	total: 10.8s	remaining: 1m 1s
400:	learn: 0.1623834	total: 14.6s	remaining: 58.2s
500:	learn: 0.1523122	total: 18.5s	remaining: 55.3s
600:	learn: 0.1444084	total: 22.5s	remaining: 52.3s
700:	learn: 0.1380168	total: 26.5s	remaining: 49s
800:	learn: 0.1327721	total: 30.5s	remaining: 45.6s
900:	learn: 0.1285837	total: 34.4s	remaining: 42s
1000:	learn: 0.1252387	total: 38.2s	remaining: 38.2s
1100:	learn: 0.1225282	total: 42.1s	remaining: 34.4s
1200:	learn: 0.1203601	total: 46s	remaining: 30.6s
1300:	learn: 0.1183225	total: 49.9s	remaining: 26.8s
1400:	learn: 0.1165826	total: 53.8s	remaining: 23s
1500:	learn: 0.1152212	total: 57.7s	remaining: 19.2s
1600:	learn: 0.1140771	total: 1m 1s	remaining: 15.4s
1700:	learn: 0.1130768	total: 1m 5s	remaining: 11.5s
1800:	learn: 0.1121519	total: 1m 9s	remaining: 7.67s
1900:	learn

In [59]:
y_pred = clf.predict(X_test)

In [60]:
d = {
    "id":ids,
    "rerata_kecepatan":y_pred
}

df = pd.DataFrame(data=d)

In [61]:
df.to_csv("Submission/cb_little_overfit.csv", index=False)